In [5]:
### forest-disturbance-stack-v3

### This script creates a function to create a custom disturbance stack. It accepts
### a template raster path (to determine resolution and extent), a list of
### individual raster file paths (e.g., biotic, wildfire, drought stacks), and a 
### forest mask path (binary mask).
### Returns and saves a final GeoTIFF (.tif).


# If working in cyverse, set working directory and project root
setwd("/home/jovyan/data-store/forest-disturbance-stack-v3")
here::i_am("README.md")   # or any file guaranteed to exist in the project



# Install and load required packages
packages <- c("here", "terra")
installed <- packages %in% installed.packages()[, "Package"]
if (any(!installed)) {
  install.packages(packages[!installed])
}

library(here)
library(terra)

# Set cyverse memory max to avoid crashing
terraOptions(memmax=256)

here() starts at /home/jovyan/data-store/forest-disturbance-stack-v3



In [ ]:
### Create template raster


# Path to a reference raster (e.g., wildfire disturbance file)
ref_raster_path <- here("data", "derived", "wildfire_id.tif")

# Load reference raster
ref <- rast(ref_raster_path)

# Define years for the template
years <- 2000:2020       # adjust to dataset

# Create a new raster at 30 m resolution with same extent & CRS as reference
template <- rast(
  extent = ext(ref),     # match extent
  resolution = 30,       # 30 m
  crs = crs(ref)         # match CRS
)

# Stack for each year
template_stack <- rast(replicate(length(years), template))
names(template_stack) <- paste0("year_", years)

# Fill with NA values so it can be written
values(template_stack) <- NA

# Save to file
output_path_temp <- here("data", "derived", "template_30m_2000_2020.tif")
writeRaster(
  template_stack,
  output_path_temp,
  datatype = "FLT4S",
  overwrite = TRUE,
  gdal = c("COMPRESS=DEFLATE")
)

message("Template saved to: ", output_path_temp)

In [12]:
### Checks before using GDAL

# Check to make sure gdalwarp is installed and available
system2("gdalwarp", args = "--version", stdout = TRUE, stderr = TRUE)

# Check to make sure gdal_calc.py is installed and available
system2("gdal_calc.py", args = "--help", stdout = TRUE, stderr = TRUE)

[1] "GDAL 3.7.2, released 2023/09/05"

[1] "usage: gdal_calc.py [--help] --calc [expression ...] [-A [filename ...]]"                                                                  
 [2] "                    [--A_band [n ...]] --outfile filename"                                                                                 
 [3] "                    [--NoDataValue value] [--hideNoData] [--type datatype]"                                                                
 [4] "                    [--format gdal_format] [--creation-option option]"                                                                     
 [5] "                    [--allBands [a-z, A-Z]] [--overwrite] [--debug] [--quiet]"                                                             
 [6] "                    [--color-table COLOR_TABLE]"                                                                                           
 [7] "                    [--extent {ignore,fail,union,intersect} | --projwin ulx uly lrx lry]"                                                  
 [8] "                    [--projectionCheck]"                                                                                                   
 [9] ""                                                                                                                                          
[10] "Raster calculator with numpy syntax"                                                                                                       
[11] "-------------------------------------"                                                                                                     
[12] "Use any basic arithmetic supported by numpy arrays such as +, -, *, and"                                                                   
[13] "along with logical operators such as >."                                                                                                   
[14] "Note that all files must have the same dimensions (unless extent option is used),"                                                         
[15] "but no projection checking is performed (unless projectionCheck option is used)."                                                          
[16] ""                                                                                                                                          
[17] "options:"                                                                                                                                  
[18] "  --help                show this help message and exit"                                                                                   
[19] "  --calc [expression ...]"                                                                                                                 
[20] "                        calculation in numpy syntax using +-/* or any numpy"                                                               
[21] "                        array functions (i.e. log10()). May appear multiple"                                                               
[22] "                        times to produce a multi-band file"                                                                                
[23] "  -A [filename ...]     input gdal raster file, you can use any letter [a-z,"                                                              
[24] "                        A-Z]"                                                                                                              
[25] "  --A_band [n ...]      number of raster band for file A (default 1)"                                                                      
[26] "  --outfile filename    output file to generate or fill"                                                                                   
[27] "  --NoDataValue value   output nodata value (default datatype specific value)"                                                             
[28] "  --hideNoData          ignores the NoDataValues of t

In [2]:
### The function

#' Resample, stack, and mask rasters (fast + multi-band safe)
#'
#' @param template_path Path to template raster (sets resolution + extent)
#' @param raster_paths Named list of rasters (wildfire, biotic, hd, pdsi, etc.)
#' @param forest_mask_path Optional forest mask raster
#' @param output_path Path for final stacked GeoTIFF
#' @param tmp_dir Temp dir for intermediate rasters
#' @param n_threads Number of threads for gdalwarp

stack_resample_mask_gdal <- function(template_path, raster_paths,
                                     forest_mask_path = NULL,
                                     output_path,
                                     tmp_dir = "gdal_tmp",
                                     n_threads = 4,
                                     gdal_calc_bin = Sys.which("gdal_calc.py")) {
  if (!dir.exists(tmp_dir)) dir.create(tmp_dir, recursive = TRUE)
  q <- function(x) shQuote(normalizePath(x, mustWork = FALSE))

  # Template info
  tmpl <- terra::rast(template_path)
  res_x <- terra::res(tmpl)[1]
  res_y <- terra::res(tmpl)[2]
  ex    <- terra::ext(tmpl)

  resampled <- character(0)

  # 1) Resample every input with gdalwarp
  for (i in seq_along(raster_paths)) {
    in_file  <- raster_paths[[i]]
    out_file <- file.path(tmp_dir, paste0(tools::file_path_sans_ext(basename(in_file)), "_", res_x, "m.tif"))
    method   <- if (grepl("wildfire|mask|id", in_file, ignore.case = TRUE)) "near" else "bilinear"

    # Skip if already present and looks OK
    if (file.exists(out_file)) {
      info <- tryCatch(system2("gdalinfo", q(out_file), stdout = TRUE, stderr = TRUE), error = function(e) "")
      if (length(grep("^Band ", info)) > 0) {
        message(sprintf("(%d/%d) Skipping %s (already resampled)", i, length(raster_paths), basename(in_file)))
        resampled <- c(resampled, out_file)
        next
      }
    }

    message(sprintf("(%d/%d) Resampling %s → %s (method=%s)",
                    i, length(raster_paths), basename(in_file), basename(out_file), method))

    cmd <- sprintf(
      "gdalwarp -tr %s %s -r %s -te %s %s %s %s -multi -wo NUM_THREADS=%d -co COMPRESS=DEFLATE %s %s",
      res_x, res_y, method,
      ex[1], ex[3], ex[2], ex[4],
      n_threads, q(in_file), q(out_file)
    )
    status <- system(cmd)
    if (status != 0) {
      warning(sprintf("gdalwarp failed for %s; skipping this raster.", basename(in_file)))
      next
    }
    resampled <- c(resampled, out_file)
  }

  if (length(resampled) == 0) stop("No rasters were successfully resampled.")

  # 2) Expand every band to its own tiny VRT, then stack them all
  band_vrts <- character(0)
  for (rf in resampled) {
    info <- system2("gdalinfo", q(rf), stdout = TRUE, stderr = TRUE)
    n_bands <- sum(grepl("^Band [0-9]+", info))
    base <- tools::file_path_sans_ext(basename(rf))
    for (b in seq_len(n_bands)) {
      b_vrt <- file.path(tmp_dir, sprintf("%s_band%03d.vrt", base, b))
      if (!file.exists(b_vrt)) {
        cmd_b <- sprintf("gdal_translate -of VRT -b %d %s %s", b, q(rf), q(b_vrt))
        status <- system(cmd_b)
        if (status != 0) warning(sprintf("gdal_translate (band %d) failed for %s; skipping that band.", b, basename(rf)))
      }
      if (file.exists(b_vrt)) band_vrts <- c(band_vrts, b_vrt)
    }
  }
  if (length(band_vrts) == 0) stop("No per-band VRTs were created.")

  message("Building virtual stack (VRT) with all bands...")
  vrt_file <- file.path(tmp_dir, "stack_bands.vrt")
  cmd_vrt <- paste("gdalbuildvrt -separate", q(vrt_file), paste(q(band_vrts), collapse = " "))
  if (system(cmd_vrt) != 0) stop("gdalbuildvrt failed.")

  # 3) Translate VRT → tiled, compressed (BigTIFF if needed)
  message("Translating VRT to GeoTIFF...")
  stacked_file <- file.path(tmp_dir, "stack.tif")
  cmd_translate <- sprintf(
    "gdal_translate %s %s -co COMPRESS=DEFLATE -co TILED=YES -co NUM_THREADS=%d -co BIGTIFF=YES",
    q(vrt_file), q(stacked_file), n_threads
  )
  if (system(cmd_translate) != 0) stop("gdal_translate failed for stack.")

  # 4) Resample forest mask to template and apply to ALL bands
  if (!is.null(forest_mask_path)) {
    resampled_mask <- file.path(tmp_dir, "forest_mask_resampled.tif")
    if (!file.exists(resampled_mask)) {
      message("Resampling forest mask...")
      cmd_mask_resample <- sprintf(
        "gdalwarp -tr %s %s -r near -te %s %s %s %s -multi -wo NUM_THREADS=%d -co COMPRESS=DEFLATE %s %s",
        res_x, res_y,
        ex[1], ex[3], ex[2], ex[4],
        n_threads,
        q(forest_mask_path), q(resampled_mask)
      )
      if (system(cmd_mask_resample) != 0) stop("gdalwarp failed for forest mask.")
    } else {
      message("Skipping forest mask resampling (already exists).")
    }

    message("Applying forest mask to ALL bands...")
    calc_bin <- if (nzchar(gdal_calc_bin)) q(gdal_calc_bin) else "gdal_calc.py"
    cmd_mask <- sprintf(
      "%s -A %s -B %s --calc=\"A*(B>0)\" --allBands=A --outfile=%s --overwrite --co=COMPRESS=DEFLATE --NoDataValue=nan",
      calc_bin, q(stacked_file), q(resampled_mask), q(output_path)
    )
    if (system(cmd_mask) != 0) stop("gdal_calc.py failed when applying mask.")
  } else {
    file.copy(stacked_file, output_path, overwrite = TRUE)
  }

  # Quick report
  fin_info <- system2("gdalinfo", q(output_path), stdout = TRUE)
  nb <- sum(grepl("^Band [0-9]+", fin_info))
  message("✅ Final stacked file saved at: ", output_path, "  (Bands: ", nb, ")")

  invisible(output_path)
}


In [6]:
stack_resample_mask_gdal <- function(template_path, raster_paths,
                                     forest_mask_path = NULL,
                                     output_path,
                                     tmp_dir = "gdal_tmp",
                                     n_threads = 4,
                                     gdal_calc_bin = Sys.which("gdal_calc.py")) {
  if (!dir.exists(tmp_dir)) dir.create(tmp_dir, recursive = TRUE)
  q <- function(x) shQuote(normalizePath(x, mustWork = FALSE))
  
  # Template info
  tmpl <- terra::rast(template_path)
  res_x <- terra::res(tmpl)[1]
  res_y <- terra::res(tmpl)[2]
  ex    <- terra::ext(tmpl)
  
  resampled <- character(0)
  
  # 1) Resample each raster
  for (i in seq_along(raster_paths)) {
    in_file  <- raster_paths[[i]]
    out_file <- file.path(tmp_dir, paste0(tools::file_path_sans_ext(basename(in_file)), "_", res_x, "m.tif"))
    method   <- if (grepl("wildfire|mask|id", in_file, ignore.case = TRUE)) "near" else "bilinear"
    
    # Skip if already present
    if (file.exists(out_file)) {
      info <- tryCatch(system2("gdalinfo", q(out_file), stdout = TRUE, stderr = TRUE), error = function(e) "")
      if (length(grep("^Band ", info)) > 0) {
        message(sprintf("(%d/%d) Skipping %s (already resampled)", i, length(raster_paths), basename(in_file)))
        resampled <- c(resampled, out_file)
        next
      }
    }
    
    message(sprintf("(%d/%d) Resampling %s → %s (method=%s)",
                    i, length(raster_paths), basename(in_file), basename(out_file), method))
    
    cmd <- sprintf(
      "gdalwarp -tr %s %s -r %s -te %s %s %s %s -multi -wo NUM_THREADS=%d -co COMPRESS=DEFLATE %s %s",
      res_x, res_y, method,
      ex[1], ex[3], ex[2], ex[4],
      n_threads, q(in_file), q(out_file)
    )
    if (system(cmd) != 0) warning(sprintf("gdalwarp failed for %s; skipping.", basename(in_file)))
    resampled <- c(resampled, out_file)
  }
  
  if (length(resampled) == 0) stop("No rasters were successfully resampled.")
  
  # 2) Create per-band VRTs
  band_vrts <- character(0)
  for (rf in resampled) {
    info <- system2("gdalinfo", q(rf), stdout = TRUE, stderr = TRUE)
    n_bands <- sum(grepl("^Band [0-9]+", info))
    base <- tools::file_path_sans_ext(basename(rf))
    for (b in seq_len(n_bands)) {
      b_vrt <- file.path(tmp_dir, sprintf("%s_band%03d.vrt", base, b))
      if (!file.exists(b_vrt)) {
        cmd_b <- sprintf("gdal_translate -of VRT -b %d %s %s", b, q(rf), q(b_vrt))
        if (system(cmd_b) != 0) warning(sprintf("gdal_translate (band %d) failed for %s", b, basename(rf)))
      }
      if (file.exists(b_vrt)) band_vrts <- c(band_vrts, b_vrt)
    }
  }
  
  if (length(band_vrts) == 0) stop("No per-band VRTs were created.")
  
  # 3) Explicitly order bands by raster type
  wildfire_vrts <- sort(grep("wildfire_id", band_vrts, value = TRUE))
  biotic_vrts   <- sort(grep("biotic_gridded", band_vrts, value = TRUE))
  pdsi_vrts     <- sort(grep("pdsi_annual", band_vrts, value = TRUE))
  hd_vrts       <- sort(grep("hd_fingerprint", band_vrts, value = TRUE))  # optional
  
  band_vrts_ordered <- c(wildfire_vrts, biotic_vrts, pdsi_vrts, hd_vrts)
  
  # 4) Build VRT
  message("Building virtual stack (VRT) with all bands in correct order...")
  vrt_file <- file.path(tmp_dir, "stack_bands.vrt")
  cmd_vrt <- paste("gdalbuildvrt -separate", q(vrt_file), paste(q(band_vrts_ordered), collapse = " "))
  if (system(cmd_vrt) != 0) stop("gdalbuildvrt failed.")
  
  # 5) Translate VRT → GeoTIFF (BigTIFF if needed)
  message("Translating VRT to GeoTIFF...")
  stacked_file <- file.path(tmp_dir, "stack.tif")
  cmd_translate <- sprintf(
    "gdal_translate %s %s -co COMPRESS=DEFLATE -co TILED=YES -co NUM_THREADS=%d -co BIGTIFF=YES",
    q(vrt_file), q(stacked_file), n_threads
  )
  if (system(cmd_translate) != 0) stop("gdal_translate failed for stack.")
  
  # 6) Apply forest mask if provided
  if (!is.null(forest_mask_path)) {
    resampled_mask <- file.path(tmp_dir, "forest_mask_resampled.tif")
    if (!file.exists(resampled_mask)) {
      message("Resampling forest mask...")
      cmd_mask_resample <- sprintf(
        "gdalwarp -tr %s %s -r near -te %s %s %s %s -multi -wo NUM_THREADS=%d -co COMPRESS=DEFLATE %s %s",
        res_x, res_y, ex[1], ex[3], ex[2], ex[4], n_threads,
        q(forest_mask_path), q(resampled_mask)
      )
      if (system(cmd_mask_resample) != 0) stop("gdalwarp failed for forest mask.")
    }
    
    message("Applying forest mask to all bands...")
    calc_bin <- if (nzchar(gdal_calc_bin)) q(gdal_calc_bin) else "gdal_calc.py"
    cmd_mask <- sprintf(
      "%s -A %s -B %s --calc=\"A*(B>0)\" --allBands=A --outfile=%s --overwrite --co=COMPRESS=DEFLATE --NoDataValue=nan",
      calc_bin, q(stacked_file), q(resampled_mask), q(output_path)
    )
    if (system(cmd_mask) != 0) stop("gdal_calc.py failed when applying mask.")
  } else {
    file.copy(stacked_file, output_path, overwrite = TRUE)
  }
  
  # 7) Report
  fin_info <- system2("gdalinfo", q(output_path), stdout = TRUE)
  nb <- sum(grepl("^Band [0-9]+", fin_info))
  message("✅ Final stacked file saved at: ", output_path, "  (Bands: ", nb, ")")
  
  invisible(output_path)
}


In [ ]:
### Usage

# Paths
template_path <- here("data", "derived", "template_30m_2000_2020.tif")

raster_paths <- c(
  here("data", "derived", "wildfire_id.tif"),
  here("data", "derived", "biotic_gridded_1km_all_years_severity.tif"),
  here("data", "derived", "pdsi_annual.tif"),
  here("data", "derived", "hd_fingerprint_corrected.tif")
)

forest_mask_path <- here("data", "derived", "relaxed_forest_mask_2000_2020.tif")

output_path <- here("data", "derived", "disturbance_stack_resampled_masked_30m.tif")

# Run function
stack_resample_mask_gdal(
  template_path = template_path,
  raster_paths = raster_paths,
  forest_mask_path = forest_mask_path,
  output_path = output_path,
  tmp_dir = "gdal_tmp",
  n_threads = 8   # override default if you want
)

(1/4) Skipping wildfire_id.tif (already resampled)

(2/4) Skipping biotic_gridded_1km_all_years_severity.tif (already resampled)

(3/4) Skipping pdsi_annual.tif (already resampled)

(4/4) Resampling hd_fingerprint_corrected.tif → hd_fingerprint_corrected_30m.tif (method=bilinear)

Warning message in stack_resample_mask_gdal(template_path = template_path, raster_paths = raster_paths, :
“gdalwarp failed for hd_fingerprint_corrected.tif; skipping.”
Warning message in system2("gdalinfo", q(rf), stdout = TRUE, stderr = TRUE):
“running command ''gdalinfo' 'gdal_tmp/hd_fingerprint_corrected_30m.tif' 2>&1' had status 1”
Building virtual stack (VRT) with all bands in correct order...

Translating VRT to GeoTIFF...



In [ ]:
### Assign layer names and re-save final stacked raster

# Load final stacked raster
stack_path <- here("data", "derived", "disturbance_stack_resampled_masked_30m.tif")
dist_stack <- rast(stack_path)

# Paths to original resampled rasters (already aligned and same resolution)
resampled_paths <- c(
  here("gdal_tmp", "wildfire_id_30m.tif"),
  here("gdal_tmp", "biotic_gridded_1km_all_years_severity_30m.tif"),
  here("gdal_tmp", "pdsi_annual_30m.tif")
)
names(resampled_paths) <- c("wildfire", "biotic", "pdsi")

# Years of interest
years_of_interest <- 2000:2020

# Number of bands in each source raster
n_bands <- sapply(resampled_paths, function(f) nlyr(rast(f)))

# Start years for each source raster
start_years <- c(
  wildfire = 1984,
  biotic   = 1997,
  pdsi     = 2000
)

# Calculate which bands correspond to 2000-2020
band_indices <- list()
current_offset <- 0
for (name in names(resampled_paths)) {
  yrs <- start_years[name]:(start_years[name] + n_bands[name] - 1)
  bands <- which(yrs %in% years_of_interest)
  # Adjust for position in the final stack
  band_indices[[name]] <- bands + current_offset
  current_offset <- current_offset + n_bands[name]
}

# Flatten to a single vector
bands_2000_2020 <- unlist(band_indices)

# Subset stack
stack_2000_2020 <- dist_stack[[bands_2000_2020]]

# Assign names
names(stack_2000_2020) <- c(
  paste0("wildfire_", years_of_interest),
  paste0("biotic_", years_of_interest),
  paste0("pdsi_", years_of_interest)
)

# Save final raster
writeRaster(
  stack_2000_2020,
  here("data", "derived", "disturbance_stack_2000_2020_30m.tif"),
  overwrite = TRUE,
  gdal = c("COMPRESS=DEFLATE")
)

message("✅ Saved final stack with only 2000-2020 bands.")
